## Import the relevant libraries

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import colours.colorsafe as cs
import seaborn as sns
import os
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import cross_validate
from sklearn.preprocessing import MinMaxScaler

## Create the results directory if it does not exist already

In [ ]:
# See https://www.tutorialspoint.com/How-can-I-create-a-directory-if-it-does-not-exist-using-Python
if not os.path.exists('res'):
  os.makedirs('res')

## Load the data into a dataframe

In [ ]:
df = pd.read_csv('data/Advertising.csv')
df.shape

In [ ]:
featureNames = df.columns[:3]
targetName = df.columns[3]
X3 = df[featureNames]
y = df[targetName]
df.head()

## Plot Sales vs. total Advertising spend

In [ ]:
# See https://stackoverflow.com/a/55654661/1988855
colours = ['trueBlue', 'orange', 'mustard']
colorSet = dict(zip(featureNames, colours))
fig, ax = plt.subplots()
ax.set_xlabel('Advertising [1000 $]')
ax.set_ylabel('Sales [Thousands of units]')
title=ax.set_title("Sales vs. Advertising", weight="bold")

# Plot Sales vs. Advertising spend per channel

for featureName in colorSet:
  color = cs.ibm5col[colorSet[featureName]]
  ax.plot(X3[featureName], y, 'o', color=color, label=featureName)

In [ ]:
# See https://stackoverflow.com/a/10129461/1988855
# ask matplotlib for the plotted objects and their labels
lines, labels = ax.get_legend_handles_labels()
ax.legend(lines, labels, loc='lower right')

In [ ]:
fig.savefig('res/data.pdf', bbox_inches='tight')
plt.show()

## Show correlation heatmap for the 3 features and 1 target variable

In [ ]:
# See https://seaborn.pydata.org/tutorial/color_palettes.html
cmap = sns.color_palette("Blues", as_cmap=True)
for method in ('pearson', 'kendall', 'spearman'):
  mCorr = df.corr(method=method)
  hm = sns.heatmap(mCorr, square=True, vmin=0, vmax=1, cmap=cmap)
  # See https://stackoverflow.com/a/47765118/1988855
  fig = hm.get_figure()
  fig.savefig('res/{}CorrHeatmap.pdf'.format(method))
  plt.title(f"{method} correlation (features and target)")
  plt.show()
  display(mCorr)

## Initialise the model and the choose the score metric

In [ ]:
model = LinearRegression(fit_intercept=True)
metric = ('neg_mean_squared_error', 'r2') # see https://scikit-learn.org/1.5/modules/model_evaluation.html#scoring-parameter

## Forward selection 1 - getting the next best feature

We identify two operations:

1. Finding the loss metric for each of a candidate set of features, and returning the feature with the largest metric
2. Accumulate the features found to date, and look for the next best feature.

The following Python function `findNextBestFeature()` provides operation 1., and the next block of python provides operation 2.

In [ ]:
def findNextBestFeature(X,foundFeatures):
  nP = X.shape[1] # number of columns in X
  allFeatures = list(X) # See https://stackoverflow.com/a/19483025
  featuresToSearch = set(allFeatures) - set(foundFeatures)
  maxScore = -np.inf # can usually do better than this!
  for feature in featuresToSearch: # loop over all remaining columns (features) in X
    trialFeatures = set(foundFeatures)
    trialFeatures.add(feature) # Add this feature to the existing features
    #display(trialFeatures)
    XcolSubset = X.loc[:,list(trialFeatures)] # all rows and just the trial features
    scores = cross_validate(model, XcolSubset, y, cv=5, scoring=metric, return_train_score=True)
    #display(scores['test_neg_mean_squared_error'])
    score = np.mean(scores['test_neg_mean_squared_error'])
    #display(score)
    trialFeatures.remove(feature) # remove the current feature from the trialFeatures, to be ready for the next candidate feature
    if score > maxScore: # identify the largest score and its associated feature
      maxScore = score
      metricsForAddedFeature = scores
      bestFeatureFound = feature
  trialFeatures.add(bestFeatureFound)
  print(f"Selected {bestFeatureFound} so current features are {trialFeatures} with score {score} and the following metrics")
  #display(metricsForAddedFeature)

  return maxScore, bestFeatureFound, metricsForAddedFeature

## Forward selection 2 - iterating over the search for the next feature to be added

Now use `findNextBestFeature` to iterate through the list of features (features)
and to keep track of the scores

In [ ]:
def prioritiseFeatures(X):
  nP = X.shape[1]
  scoreHistory = dict()
  foundFeatures = list()

  for i in range(nP): # loop over all columns (features) in X
    score, bestFeatureFound, metricsForAddedFeature = findNextBestFeature(X, foundFeatures)
    foundFeatures.append(bestFeatureFound)
    scoreHistory[bestFeatureFound] = metricsForAddedFeature

  #display(foundFeatures)
  #display(scoreHistory)
  return foundFeatures, scoreHistory

## Preparing a dataframe to make feature analysis easier

Parse the scores and map into a dataframe for convenient analysis

In [ ]:
def parseToDF(foundFeatures, scoreHistory):
  scoreList = []
  for feature in foundFeatures:
    scoreDS = scoreHistory[feature]
    fitTime = np.mean(scoreDS["fit_time"])
    scoreTime = np.mean(scoreDS["score_time"])
    testNegMSE = (np.min(scoreDS["test_neg_mean_squared_error"]), np.max(scoreDS["test_neg_mean_squared_error"]))
    trainNegMSE = (np.min(scoreDS["train_neg_mean_squared_error"]), np.max(scoreDS["train_neg_mean_squared_error"]))
    testR2 = (np.min(scoreDS["test_r2"]), np.max(scoreDS["test_r2"]))
    trainR2 = (np.min(scoreDS["train_r2"]), np.max(scoreDS["train_r2"]))
    record = {'feature': feature, 'fit_time': fitTime, 'score_time': scoreTime,
              'test_neg_mean_squared_error': testNegMSE, 'train_neg_mean_squared_error': trainNegMSE,
              'test_r2': testR2, 'train_r2': trainR2}
    scoreList.append(record)
  
  scoresDf = pd.DataFrame.from_dict(scoreList)
  return scoresDf

## Derive the scores dataframe for a given feature matrix X

In [ ]:
def deriveScoresDf(X):
  foundFeatures, scoreHistory = prioritiseFeatures(X)
  scoresDf = parseToDF(foundFeatures, scoreHistory)
  return scoresDf

In [ ]:
scoresDf3 = deriveScoresDf(X3)
display(scoresDf3[["feature", "test_neg_mean_squared_error", "test_r2"]])

## Analysis

Note that there is a lot of overlap between the test scores for `TV + Radio` and `TV + Radio + Newspaper`
(the minimum and maximum for each class of metric are very similar). Hence, `Newspaper` adds almost
no value to the model, so we do not include this feature.

## Interaction term

To add an interaction term, we need to multiply the two features in that interaction. However, this can
introduce scaling difficulties, so we scale the features so that all features (including the new
interaction term) will have the same scaling.
Doing the scaling after multiplication makes it easier to undo the scaling afterwards, if needed.

In [ ]:
X4 = X3.copy()
X4["TV:Radio"] = X4['TV'] * X4['Radio']
scaler = MinMaxScaler()
scaler.fit(X4)
X4scaled = scaler.transform(X4)

## Generate the scores for X4 (with the interaction term)

In [ ]:
scoresDf4 = deriveScoresDf(X4)
display(scoresDf4[["feature", "test_neg_mean_squared_error", "test_r2"]])

## Analysis

The `TV:Radio` feature is a more significant addition to the model than `Radio` on its own.
Using the same criterion as before, we can surmise that the best model is `TV + TV:Radio`.

## Exercise: Reproduce this notebook using `statsmodels` instead of `scikit-learn`.